In [ ]:
# =====================================================
# 🌦️ 5 European Cities Weather Fetcher + CSV Export
# =====================================================

import requests
import pandas as pd
from datetime import datetime

# --- User Input ---
print("Enter city names (comma-separated from: Paris, Vienna, Prague, Rome, Lisbon):")
city_input = input("Cities: ").split(",")
city_input = [c.strip().title() for c in city_input if c.strip()]

print("\nEnter start date (YYYY-MM-DD):")
start_date = input("Start date: ")

print("Enter end date (YYYY-MM-DD):")
end_date = input("End date: ")

# --- Coordinates for 5 European Cities ---
city_coords = {
    "Paris": (48.8566, 2.3522),
    "Vienna": (48.2082, 16.3738),
    "Prague": (50.0755, 14.4378),
    "Rome": (41.9028, 12.4964),
    "Lisbon": (38.7169, -9.1399)
}

# --- API URL ---
url = "https://api.open-meteo.com/v1/forecast"

# --- Fetch Function ---
def get_weather_data(city, lat, lon, start_date, end_date):
    params = {
        "latitude": lat,
        "longitude": lon,
        "hourly": [
            "temperature_2m",
            "relative_humidity_2m",
            "pressure_msl",
            "wind_speed_10m",
            "wind_direction_10m",
            "precipitation",
            "snowfall",
            "snow_depth",
            "visibility"
        ],
        "start_date": start_date,
        "end_date": end_date,
        "timezone": "auto"
    }

    r = requests.get(url, params=params)
    r.raise_for_status()
    data = r.json().get("hourly", {})

    df = pd.DataFrame(data)
    df["city"] = city
    return df

# --- Collect Data ---
all_data = []
for city in city_input:
    if city in city_coords:
        lat, lon = city_coords[city]
        print(f"Fetching weather data for {city}...")
        df = get_weather_data(city, lat, lon, start_date, end_date)
        all_data.append(df)
    else:
        print(f"⚠️ Skipping '{city}': Not in coordinate list.")

if not all_data:
    raise ValueError("No valid cities entered. Please pick from Paris, Vienna, Prague, Rome, Lisbon.")

combined_df = pd.concat(all_data, ignore_index=True)

# --- Standardize Columns ---
final_df = combined_df.rename(columns={
    "time": "date_time",
    "temperature_2m": "temperature",
    "relative_humidity_2m": "humidity",
    "pressure_msl": "pressure",
    "wind_speed_10m": "wind_speed",
    "wind_direction_10m": "wind_direction"
})

final_df["station_id"] = final_df["city"] + "_01"

final_df = final_df[
    ["date_time", "station_id", "city", "temperature", "humidity", "pressure",
     "wind_speed", "wind_direction", "precipitation", "snowfall", "snow_depth", "visibility"]
]

# --- Save CSV ---
filename = f"/content/europe_weather_data_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
final_df.to_csv(filename, index=False)

print("\n✅ Data successfully saved to:", filename)
print("Preview of your data:")
display(final_df.head())

# --- Optional: Download CSV to your system ---
from google.colab import files
files.download(filename)

Enter city names (comma-separated from: Paris, Vienna, Prague, Rome, Lisbon):
Cities: Paris, Vienna, Prague, Rome, Lisbon

Enter start date (YYYY-MM-DD):
Start date: 2025-10-01
Enter end date (YYYY-MM-DD):
End date: 2025-11-03
Fetching weather data for Paris...
Fetching weather data for Vienna...
Fetching weather data for Prague...
Fetching weather data for Rome...
Fetching weather data for Lisbon...

✅ Data successfully saved to: /content/europe_weather_data_20251104_051737.csv
Preview of your data:


,date_time,station_id,city,temperature,humidity,pressure,wind_speed,wind_direction,precipitation,snowfall,snow_depth,visibility
0,2025-10-01T00:00,Paris_01,Paris,12.8,72,1027.0,4.0,333,0.0,0.0,0.0,49900.0
1,2025-10-01T01:00,Paris_01,Paris,12.3,76,1027.3,4.8,347,0.0,0.0,0.0,41560.0
2,2025-10-01T02:00,Paris_01,Paris,11.5,80,1026.9,4.7,337,0.0,0.0,0.0,31620.0
3,2025-10-01T03:00,Paris_01,Paris,11.1,83,1027.0,4.5,331,0.0,0.0,0.0,25520.0
4,2025-10-01T04:00,Paris_01,Paris,10.8,85,1027.1,3.7,331,0.0,0.0,0.0,22160.0


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>